# Loading Model


Install required Libraries

In [1]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-ta2h__h1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-ta2h__h1
  Resolved https://github.com/huggingface/accelerate to commit 80da9cfb09bb3cc9f1b385cb55d6b90d025a5fd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-none-any.whl size=258139 sha256=4d567dfa55da7902d09d043b9a08ba4c6e06bd7d9af9fd30a9499c5

Load the libraries and Load the Tokenizer

In [2]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-rw-1b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Build the Model Pipeline using Hugging Face Transformers Pipeline

In [3]:
from peft import LoraConfig

lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


# Dataset


In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead

dataset_name = "pubmed_qa"
dataset = load_dataset(dataset_name, "pqa_artificial")
#Format the input
tokenizer_sum=AutoTokenizer.from_pretrained('T5-small')
model_sum=AutoModelWithLMHead.from_pretrained('T5-small', return_dict=True)

dataset_reformat = dataset['train'].train_test_split(test_size=9000, train_size=21000)

def combine_context(row):
    row['context'] = ''.join(row['context']['contexts'])
    words = torch.split(row['context'], " ")
    word_count = len(words)
    if word_count > 512:
      inputs=tokenizer_sum.encode("sumarize: " +row['context'],return_tensors='pt', max_length=512, truncation=True)
      output = model_sum.generate(inputs, min_length=80, max_length=512)
      summary=tokenizer_sum.decode(output[0])
      row['context'] = summary
    return row

def get_text(row):
  row['text'] = "### Context: " + row['context'] +  "### Question: " + row['question'] + "### Answer: " + row['long_answer']
  return row

dataset_train = dataset_reformat['train']
dataset_train.add_column(name="text", column=dataset_train['long_answer'])
dataset_train = dataset_train.map(combine_context)
for i in range(10):
  print(dataset_train['context'][i])
dataset_train = dataset_train.map(get_text,  remove_columns=dataset_reformat["train"].column_names)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/211269 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/21000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

This study examined the protein and messenger RNA (mRNA) expression of molecules associated with M2 (wound healing) macrophages in mineral trioxide aggregate (MTA)-implanted rat subcutaneous tissue to elucidate the involvement of M2 macrophages in the connective tissue response to MTA.Silicone tubes containing freshly mixed MTA or a calcium hydroxide cement (Life; Kerr, Romulus, MI) were subcutaneously implanted into the backs of Wistar rats. Solid silicone rods implanted in different animals served as controls. The specimens were then double immunostained for ED1 (CD68, a general macrophage marker) and ED2 (CD163, an M2 macrophage marker). Immunostaining for CD34 (a marker for vascularization and wound healing) was also performed. Expression levels of CD34, CD163, and mannose receptor c type 1 (an M2 macrophage marker) mRNAs were determined with real-time polymerase chain reaction.MTA-implanted subcutaneous tissues showed significant increases in the density of ED1+ED2+ macrophages be

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

In [ ]:
print(dataset_train)

Dataset({
    features: ['text'],
    num_rows: 21000
})


# Loading the trainer

In [6]:
from transformers import TrainingArguments



training_arguments = TrainingArguments(

    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs = 1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    max_steps = 500,
    optim = "paged_adamw_32bit",
    output_dir = "./results",
    lr_scheduler_type = "constant",
    warmup_ratio = 0.03,
    max_grad_norm = 0.3,
    save_steps = 10,
    report_to=None,
    group_by_length=True

)



Pass everything to Trainer

In [7]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

pre-process the model by upcasting the layer norms in float 32 for more stable training

In [8]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)



# Train The Model

In [9]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
trainer.train()

wandb: Currently logged in as: hungnguyennguyen200504. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.10
wandb: Run data is saved locally in /content/wandb/run-20230921_171959-4nwvc1rc
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run breezy-fog-26
wandb: ⭐️ View project at https://wandb.ai/hungnguyennguyen200504/huggingface
wandb: 🚀 View run at https://wandb.ai/hungnguyennguyen200504/huggingface/runs/4nwvc1rc
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.873800
10,1.810100
15,1.825000
20,1.858400
25,1.913500
30,1.894100
35,1.981300
40,2.023500
45,1.997800
50,2.156400


TrainOutput(global_step=500, training_loss=1.8975086746215821, metrics={'train_runtime': 3593.9915, 'train_samples_per_second': 2.226, 'train_steps_per_second': 0.139, 'total_flos': 1.176778327425024e+16, 'train_loss': 1.8975086746215821, 'epoch': 0.38})